In [1]:
import sys
sys.path.append("../")

In [45]:
import os
import jsonlines
import asyncio

from pydriller import RepositoryMining, Commit
from pydriller.domain.commit import Modification

from src.download_diffs import DiffMiner

from concurrent.futures.thread import ThreadPoolExecutor

In [27]:
repository_path = "/tmp/repositories/ansible_ansible"

In [28]:
miner = RepositoryMining(
    path_to_repo=repository_path
)

In [29]:
commits = list(miner.traverse_commits())

In [40]:
def _parse_modification(modification):
    return {
        "added": modification.added,
        "removed": modification.removed,
        "change_type": modification.change_type.name,
        "diff": modification.diff,
        "filename": modification.filename,
        "old_path": modification.old_path,
        "new_path": modification.new_path,
    }

def _parse_commit(commit):
    return {
        "sha": commit.hash,
        "repository": commit.project_name,
        "message": commit.msg,
        "modifications": [
            _parse_modification(modification)
            for modification in commit.modifications
        ],
    }

In [43]:
%%time
parsed_commits = [_parse_commit(commit) for commit in commits[1000:2000]]

CPU times: user 2.53 s, sys: 5.93 s, total: 8.46 s
Wall time: 24.3 s


In [47]:
x = [1,2,3]

In [48]:
x[2:10]

[3]

In [50]:
def parse_data_chunk(commits):
    return [_parse_commit(commit) for commit in commits]

async def main():
    step = 100
    commits = list(miner.traverse_commits())[:1000]
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor(max_workers=20) as executor:
        futures = [loop.run_in_executor(executor, parse_data_chunk, commits[i:i+step]) for i in range(len(commits), step=step)]
        parsed_commits = [await res for res in asyncio.as_completed(futures)]
        return parsed_commits

In [51]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop